In [2]:
#!pip install minsearch

In [14]:
import minsearch, json

In [15]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [16]:
#docs_raw

In [17]:
documents=[]
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [18]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
index = minsearch.Index(
    text_fields=['question','section','text'],
    keyword_fields=['course']
)

In [20]:
index.fit(documents)

In [21]:
q = 'course already started, can i still enroll'

In [29]:
boost = {'question':3.0, 'text':0.5}

results = index.search(
    query=q,
    boost_dict=boost,
    num_results = 10
)

In [30]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [31]:
from google import genai
import os
os.environ["GOOGLE_API_KEY"] = "YOUR KEY HERE"

In [32]:
client = genai.Client()

In [33]:
response = client.models.generate_content(
            model="gemini-2.0-flash", 
            contents="Explain how AI works in a few words?"
        )
print(response.text)

AI learns from data to make predictions or decisions.



In [52]:
prompt_template = """
You are a course teaching assistant. 
Answer the QUESTION based on the CONTEXT from the FAQ document
Use only the facts from the CONTEXT while answering the QUESTION
If the CONTEXT does not contain the answer, output NONE
QUESTION: {question}
CONTEXT: {context}

""".strip()

In [53]:
context =""
for doc in results:
    context = context + f"section: {doc['section']}\n question: {doc['question']}\n answer: {doc['text']}\n\n " 

In [67]:
# q = "I want a beer" #check for none
q = 'course already started, can i still enroll'

In [68]:
#print(context)
prompt = prompt_template.format(question = q, context =context).strip()

In [69]:
print(prompt)

You are a course teaching assistant. 
Answer the QUESTION based on the CONTEXT from the FAQ document
Use only the facts from the CONTEXT while answering the QUESTION
If the CONTEXT does not contain the answer, output NONE
QUESTION: course already started, can i still enroll
CONTEXT: section: General course-related questions
 question: The course has already started. Can I still join it?
 answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

 section: General course-related questions
 question: Course - Can I still join the course after the start date?
 answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that t

In [73]:
#lets do it in gemini
response = client.models.generate_content(
            model="gemini-2.0-flash", 
            contents= prompt 
        )
print(response.text)

Yes, you can still join the course even after it has started. You won’t be able to submit some of the homeworks, but you can still take part in the course. You can still submit the homeworks even if you don't register.
To get a certificate, submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline.

